In [6]:
# Enable interactive backend for pick events
# Prefer ipympl in JupyterLab; fallback to notebook; else warn
try:
    get_ipython().run_line_magic('matplotlib', 'widget')
    print('Using %matplotlib widget')
except Exception:
    try:
        get_ipython().run_line_magic('matplotlib', 'notebook')
        print('Using %matplotlib notebook')
    except Exception:
        import matplotlib
        print('Interactive backend not available. Current backend:', matplotlib.get_backend())
        print('Install ipympl: pip install ipympl, then restart the kernel.')



Using %matplotlib widget


In [7]:
import os
import re
import glob
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import ipywidgets as widgets
import time



In [8]:
# Assumes these arrays are available or load from npz/csv
# Here we try to load from the same npz created earlier
latent = None
f_values = None
k_values = None

npz_path = os.path.join(os.path.dirname(__file__), 'cluster_0_data.npz') if '__file__' in globals() else 'cluster_0_data.npz'
if os.path.exists(npz_path):
    data = np.load(npz_path)
    f_values = data['f']
    k_values = data['k']
else:
    # fallback if needed
    f_csv = os.path.join(os.path.dirname(__file__), 'cluster_0_f_values.csv') if '__file__' in globals() else 'cluster_0_f_values.csv'
    k_csv = os.path.join(os.path.dirname(__file__), 'cluster_0_k_values.csv') if '__file__' in globals() else 'cluster_0_k_values.csv'
    f_values = np.loadtxt(f_csv, delimiter=',')
    k_values = np.loadtxt(k_csv, delimiter=',')

f_values = np.asarray(f_values, dtype=float).reshape(-1)
k_values = np.asarray(k_values, dtype=float).reshape(-1)
assert f_values.shape == k_values.shape

# Prepare GIF index from directory for robust lookup
# Match pattern: GrayScott-f0.0100-k0.0500-*.gif
pattern_re = re.compile(r"GrayScott-f([0-9]+\.[0-9]+)-k([0-9]+\.[0-9]+)-.*\\.gif$")

GIF_DIR = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), 'data', 'gif'))
if not os.path.exists(GIF_DIR):
    GIF_DIR = os.path.abspath(os.path.join(os.getcwd(), 'data', 'gif'))

all_gifs = sorted(glob.glob(os.path.join(GIF_DIR, 'GrayScott-f*-k*-*.gif')))
avail_f = []
avail_k = []
avail_paths = []
for p in all_gifs:
    m = pattern_re.search(p)
    if not m:
        continue
    try:
        f_parsed = float(m.group(1))
        k_parsed = float(m.group(2))
    except Exception:
        continue
    avail_f.append(f_parsed)
    avail_k.append(k_parsed)
    avail_paths.append(p)

avail_f = np.asarray(avail_f, dtype=float).reshape(-1)
avail_k = np.asarray(avail_k, dtype=float).reshape(-1)



In [9]:
# GIF preview output box - fixed size to prevent layout shifts
output_box = widgets.Output(layout=widgets.Layout(width='400px', height='400px', overflow='hidden', border='1px solid #ccc', flex='0 0 auto'))


In [10]:
# Build scatter plot (f-k map) with pick events
plt.ioff()  # Turn off automatic figure display
fig, ax = plt.subplots(figsize=(6, 5))
sc = ax.scatter(k_values, f_values, s=24, alpha=0.7, picker=5)
ax.set_xlabel('k')
ax.set_ylabel('f')
ax.set_title('Click a point to preview GIF (data/gif)')
ax.grid(True, alpha=0.3)
fig.tight_layout()  # Prevent label clipping

# Small epsilon to match floating-point formatted filenames
def format_fk_for_filename(f: float, k: float) -> str:
    # Examples: GrayScott-f0.0100-k0.0500-*.gif
    return f"GrayScott-f{f:.4f}-k{k:.4f}-*.gif"

# Find candidate GIFs under data/gif
GIF_DIR = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), 'data', 'gif'))
if not os.path.exists(GIF_DIR):
    # fallback if running from repo root
    GIF_DIR = os.path.abspath(os.path.join(os.getcwd(), 'data', 'gif'))

print('GIF directory:', GIF_DIR)

last_annotation = None

# helper to find nearest available gif by rounding
# we try the exact format first; if not found, we snap to nearest (f,k) present in filenames

def find_gif_for_fk(f: float, k: float):
    # exact match
    pattern = format_fk_for_filename(f, k)
    search_path = os.path.join(GIF_DIR, pattern)
    matches = sorted(glob.glob(search_path))
    if matches:
        return matches[0]
    # nearest neighbor search in available parsed list
    if avail_f.size > 0:
        d2 = (avail_f - f)**2 + (avail_k - k)**2
        j = int(np.argmin(d2))
        return avail_paths[j]
    return None


def on_pick(event):
    global last_annotation
    if last_annotation is not None:
        last_annotation.remove()
        last_annotation = None

    ind = event.ind
    if len(ind) == 0:
        return
    idx = ind[0]
    f = float(f_values[idx])
    k = float(k_values[idx])

    # annotate clicked point
    last_annotation = ax.annotate(f"(f={f:.4f}, k={k:.4f})", (k, f),
                                  textcoords="offset points", xytext=(10, 10),
                                  bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.6))
    fig.canvas.draw_idle()

    # resolve gif path (exact or nearest)
    gif_path = find_gif_for_fk(f, k)
    if not gif_path or not os.path.exists(gif_path):
        print('GIF not found near', (f, k))
        return

    print('Displaying:', gif_path)
    # show only one output area (clears previous) - always in the same location
    with output_box:
        clear_output(wait=True)
        display(Image(filename=gif_path, embed=True, width=400))


# Size the interactive canvas to avoid overlap
fig.canvas.layout = widgets.Layout(width='550px', height='450px', border='1px solid #ccc', flex='0 0 auto')

# Display plot and GIF output side-by-side
container = widgets.HBox(
    [fig.canvas, output_box],
    layout=widgets.Layout(align_items='flex-start', justify_content='flex-start', gap='12px', overflow='visible')
)

# Clear all previous outputs before displaying to prevent duplication
clear_output(wait=False)
display(container)

cid = fig.canvas.mpl_connect('pick_event', on_pick)
